<a href="https://colab.research.google.com/github/JoshuaHaga/Covid_19_sprint_1_buildwk/blob/master/Covid_19_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
### Must run cell below to load data ###

In [87]:
!wget 'https://storage.googleapis.com/kaggle-data-sets/555089%2F1416097%2Fbundle%2Farchive.zip?GoogleAccessId=gcp-kaggle-com@kaggle-161607.iam.gserviceaccount.com&Expires=1598147753&Signature=IukcSTm6w5zUlHaMV4oEP54bDH7VN6pQZQ5IXC5lPv0iVDEdglpdZ3muSmSpuCc56KZKg4zErk2hCG5BocMkfIwGNamqjqJtdHJFKdL4LC2jU%2B71KL%2BVFwfpv6q907o7MztOCZ44oC444RnrTDiobim5%2FdcBZAJ2GYLKpV3TdavOVMtWxhnWDs1r0BhwLLlWFINNa5eRfpmSn9n5MDSkBwZGq9nmmNMVjJHRv%2FRTfflREET2aTC8hPBFSQQN57Wk6HBvWzSzWrbxv91qZ0RU8EY%2BocXQ7jjshUxJzNr1%2BAqlmSIEB9azuFGzUw0JeJoXzbLjS59PLj3R0k2TMEUb%2Fg%3D%3D'
!unzip '/content/555089%2F1416097%2Fbundle%2Farchive.zip?GoogleAccessId=gcp-kaggle-com@kaggle-161607.iam.gserviceaccount.com&Expires=1598147753&Signature=IukcSTm6w5zUlHaMV4oEP54bDH7VN6pQZQ5IXC5lPv0iVDEdglpdZ3muSmSpuCc56KZKg4zErk2hCG5BocMkfIwGNamqjqJtdHJ'

The name is too long, 487 chars total.
Trying to shorten...
New name is 555089%2F1416097%2Fbundle%2Farchive.zip?GoogleAccessId=gcp-kaggle-com@kaggle-161607.iam.gserviceaccount.com&Expires=1598147753&Signature=IukcSTm6w5zUlHaMV4oEP54bDH7VN6pQZQ5IXC5lPv0iVDEdglpdZ3muSmSpuCc56KZKg4zErk2hCG5BocMkfIwGNamqjqJtdHJ.
--2020-08-21 03:20:32--  https://storage.googleapis.com/kaggle-data-sets/555089%2F1416097%2Fbundle%2Farchive.zip?GoogleAccessId=gcp-kaggle-com@kaggle-161607.iam.gserviceaccount.com&Expires=1598147753&Signature=IukcSTm6w5zUlHaMV4oEP54bDH7VN6pQZQ5IXC5lPv0iVDEdglpdZ3muSmSpuCc56KZKg4zErk2hCG5BocMkfIwGNamqjqJtdHJFKdL4LC2jU%2B71KL%2BVFwfpv6q907o7MztOCZ44oC444RnrTDiobim5%2FdcBZAJ2GYLKpV3TdavOVMtWxhnWDs1r0BhwLLlWFINNa5eRfpmSn9n5MDSkBwZGq9nmmNMVjJHRv%2FRTfflREET2aTC8hPBFSQQN57Wk6HBvWzSzWrbxv91qZ0RU8EY%2BocXQ7jjshUxJzNr1%2BAqlmSIEB9azuFGzUw0JeJoXzbLjS59PLj3R0k2TMEUb%2Fg%3D%3D
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.217.128, 172.217.204.128, 172.217.203.128, ...
C

##COVID-19 in USA

#Context
Data is obtained from COVID-19 Tracking project and NYTimes. Sincere thanks to them for making it available to the public.

Coronaviruses are a large family of viruses which may cause illness in animals or humans. In humans, several coronaviruses are known to cause respiratory infections ranging from the common cold to more severe diseases such as Middle East Respiratory Syndrome (MERS) and Severe Acute Respiratory Syndrome (SARS). The most recently discovered coronavirus causes coronavirus disease COVID-19 - World Health Organization

The number of new cases are increasing day by day around the world. This dataset has information from 50 US states and the District of Columbia at daily level.

The Original Dataset can be found on kaggle by clicking [here](https://www.kaggle.com/sudalairajkumar/covid19-in-usa).

#Index
All datasets, functions, libraries, and variables used throughout the notebook  will be outlined here. 

###Libraries

In [88]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


###Datasets

In [89]:
### These 3 dataframes come from the kaggle dataset 'COVID-19 in USA'
counties = pd.read_csv('/content/us_counties_covid19_daily.csv')
states   = pd.read_csv('/content/us_states_covid19_daily.csv')
us       = pd.read_csv('/content/us_states_covid19_daily.csv')

###Functions

In [90]:
### Checks for null values. 
def null_check(dataframe):
    val = dataframe.isnull().sum()
    val_sum = val.sum()
    if val_sum == 0:
      print('Zero null values.')
    else:
      print(val)

### Finds null counts, shape, describes and previews data set. 
def explore(dataframe):
  print(null_check(dataframe))
  print('Shape: ', dataframe.shape)
  print(dataframe.describe())
  return dataframe.head()

### Creates a list from column names
def col_list(dataframe):
  val_counts = dataframe.value_counts()
  val_as_pd  = pd.DataFrame(val_counts).T
  val_list   = val_as_pd.columns.tolist()
  val_pd     = pd.DataFrame(val_list, columns=['county'])
  return val_pd

### Calculates daily averages of new cases for counties. Outputs as pandas df
def cnty_daily_avg(dataframe, countyname):
  county = dataframe[dataframe['county']==countyname]
  mean   = county['cases'].mean()
  avg    = mean/len(county)
  dat    = {'county_name':[countyname],'avg_daily':[avg]}
  df     = pd.DataFrame(dat)
  return df

###Applies county daily average function across dataframe from a list. 
def daily_df(dataframe, countylist,):
  data_list = []
  for i in countylist:
    data_list.append(cnty_daily_avg(dataframe, i)) 
  return pd.DataFrame(data_list, columns=['county','daily_avg'])

###Variables

In [161]:

################################################################################
###########################<<---- Maryland ---->>###############################
################################################################################
###  md_counties                 #< - - - - - - - - - - - All Counties in MD ###
###  md_counties_vc              #< - - - - - Value counts of counties in MD ###
###  md_county_names             #< - - - - - - - - - List of counties in MD ###
###  baltimore_daily_average     #< - - - - Average of new cases daily in MD ###
###  md_daily_avg                #< - - Daily averages of all counties in MD ### 

# Data Exploration

### Introduction:
  Here I will. be doing some basic data explorations of the sets listed in the index. I will clean data here and begin to draw conclusions to be processed later. Findings will be collected and expounded upon at the end of this section. 

##Counties 
Here I am going to explore the counties data set and see what all I can find. Since I live in MD, I am going to first take a look at the number of positive cases since the start of data collection

In [ ]:
### Doing some initial exploration in the counties dataset ###
explore(counties)

In [ ]:
md_counties = counties[counties['state']=='Maryland']
md_counties

In [537]:
### Checking fo null data 
null_check(md_counties)

date        0
county      0
state       0
fips      122
cases       0
deaths      0
dtype: int64


In [ ]:
##### Counts of counties shows pretty even sampling across counties. 
md_counties_vc = md_counties['county'].value_counts()
md_counties_vc

In [ ]:
#Creating a list of county names in md
mdcounty_list = pd.DataFrame(md_counties_vc)
mdcounty_list = mdcounty_list.T
mdcounty_list

In [ ]:
md_county_names2 = mdcounty_list.columns.tolist()
md_county_names2

In [498]:
#The whole process as a function to speed up later states. 
def col_list(dataframe):
  val_counts = dataframe.value_counts()
  val_as_pd  = pd.DataFrame(val_counts).T
  val_list   = val_as_pd.columns.tolist()
  val_pd     = pd.DataFrame(val_list, columns=['county'])
  return val_list

In [ ]:
md_county_names = col_list(md_counties['county']) ###<<< - - - - Testing listing function
md_county_names

In [ ]:
#Finding average cases per county during the entire time of recording
#Starting with Baltimore

In [ ]:
baltimore = md_counties[md_counties['county']=='Baltimore']
baltimore

In [174]:
balt_mean = baltimore['cases'].mean()

In [500]:
baltimore_daily_average = balt_mean/len(baltimore)
baltimore_daily_average

35.56853277835588

In [532]:
### Create function to do this faster for later 

def cnty_daily_avg(dataframe, countyname):
  county = dataframe[dataframe['county']==countyname]
  mean   = county['cases'].mean()
  avg    = mean/len(county)
  dat    = [countyname,avg]
  return dat


In [533]:
balt_avg = cnty_daily_avg(md, 'Baltimore') #< - - - thank god I dont have to do this hundreds of times manually
balt_avg

['Baltimore', 35.56853277835588]

In [534]:
#Apply function to all counties and create a dataframe
data_list = []

In [535]:
for i in md_county_names:
  data_list.append(cnty_daily_avg(md, i)) 

In [ ]:
data_list

In [526]:
###create function to do same thing. 
def daily_df(dataframe, countylist,):
  data_list = []
  for i in countylist:
    data_list.append(cnty_daily_avg(dataframe, i)) 
  return pd.DataFrame(data_list, columns=['county','daily_avg'])

In [531]:
md_daily_avg = daily_df(md, md_county_names,)
md_daily_avg.head()

,county,daily_avg
0,Montgomery,55.658115
1,Harford,4.924651
2,Prince George's,77.005233
3,Baltimore,35.568533
4,Anne Arundel,22.200991


##States

In [95]:
explore(states)

date                           0
state                          0
positive                      15
negative                     156
pending                     7859
hospitalizedCurrently       2618
hospitalizedCumulative      4088
inIcuCurrently              5654
inIcuCumulative             7642
onVentilatorCurrently       6092
onVentilatorCumulative      8503
recovered                   3275
dataQualityGrade            1157
lastUpdateEt                 343
dateModified                 343
checkTimeEt                  343
death                        698
hospitalized                4088
dateChecked                  343
totalTestsViral             5375
positiveTestsViral          7772
negativeTestsViral          7737
positiveCasesViral          3087
deathConfirmed              5780
deathProbable               6885
fips                           0
positiveIncrease               0
negativeIncrease               0
total                          0
totalTestResults               0
totalTestR

,date,state,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,onVentilatorCumulative,recovered,dataQualityGrade,lastUpdateEt,dateModified,checkTimeEt,death,hospitalized,dateChecked,totalTestsViral,positiveTestsViral,negativeTestsViral,positiveCasesViral,deathConfirmed,deathProbable,fips,positiveIncrease,negativeIncrease,total,totalTestResults,totalTestResultsIncrease,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20200812,AK,4655.0,287927.0,NaN,39.0,NaN,NaN,NaN,3.0,NaN,1359.0,NaN,8/12/2020 00:00,2020-08-12T00:00:00Z,08/11 20:00,27.0,NaN,2020-08-12T00:00:00Z,292582.0,4646.0,287633.0,4655.0,27.0,NaN,2,66,1772,292582,292582,1838,292582,1,0,04d3f681722d466f15660e1ded53888693484520,0,0,0,0,0,NaN
1,20200812,AL,104786.0,690985.0,NaN,1372.0,12292.0,NaN,1282.0,NaN,700.0,41523.0,NaN,8/12/2020 11:00,2020-08-12T11:00:00Z,08/12 07:00,1882.0,12292.0,2020-08-12T11:00:00Z,791786.0,NaN,NaN,100801.0,1814.0,68.0,1,935,11147,795771,795771,12082,795771,35,222,3b8e75d37adb9ba3951ec8b2aae85ba1fa9b02f1,0,0,0,0,0,NaN
2,20200812,AR,51114.0,522457.0,NaN,486.0,3472.0,NaN,NaN,113.0,455.0,42998.0,NaN,8/12/2020 14:40,2020-08-12T14:40:00Z,08/12 10:40,566.0,3472.0,2020-08-12T14:40:00Z,572868.0,NaN,522457.0,51114.0,NaN,NaN,5,703,0,573571,573571,703,573571,0,71,614656008210925b859900619d0426d793e79029,0,0,0,0,0,NaN
3,20200812,AS,0.0,1396.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7/24/2020 00:00,2020-07-24T00:00:00Z,07/23 20:00,0.0,NaN,2020-07-24T00:00:00Z,NaN,NaN,NaN,0.0,NaN,NaN,60,0,0,1396,1396,0,1396,0,0,f6988932e32e01718e47c8647131e3311e70d481,0,0,0,0,0,NaN
4,20200812,AZ,189443.0,854785.0,NaN,1469.0,19821.0,519.0,NaN,328.0,NaN,26418.0,NaN,8/12/2020 00:00,2020-08-12T00:00:00Z,08/11 20:00,4347.0,19821.0,2020-08-12T00:00:00Z,1042838.0,NaN,NaN,188053.0,4095.0,252.0,4,706,6247,1044228,1044228,6953,1044228,148,529,83481965d3dca76744e02ade2b8ec71c6fec31b7,0,0,0,0,0,NaN


##US


In [96]:
explore(us)

date                           0
state                          0
positive                      15
negative                     156
pending                     7859
hospitalizedCurrently       2618
hospitalizedCumulative      4088
inIcuCurrently              5654
inIcuCumulative             7642
onVentilatorCurrently       6092
onVentilatorCumulative      8503
recovered                   3275
dataQualityGrade            1157
lastUpdateEt                 343
dateModified                 343
checkTimeEt                  343
death                        698
hospitalized                4088
dateChecked                  343
totalTestsViral             5375
positiveTestsViral          7772
negativeTestsViral          7737
positiveCasesViral          3087
deathConfirmed              5780
deathProbable               6885
fips                           0
positiveIncrease               0
negativeIncrease               0
total                          0
totalTestResults               0
totalTestR

,date,state,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,onVentilatorCumulative,recovered,dataQualityGrade,lastUpdateEt,dateModified,checkTimeEt,death,hospitalized,dateChecked,totalTestsViral,positiveTestsViral,negativeTestsViral,positiveCasesViral,deathConfirmed,deathProbable,fips,positiveIncrease,negativeIncrease,total,totalTestResults,totalTestResultsIncrease,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20200812,AK,4655.0,287927.0,NaN,39.0,NaN,NaN,NaN,3.0,NaN,1359.0,NaN,8/12/2020 00:00,2020-08-12T00:00:00Z,08/11 20:00,27.0,NaN,2020-08-12T00:00:00Z,292582.0,4646.0,287633.0,4655.0,27.0,NaN,2,66,1772,292582,292582,1838,292582,1,0,04d3f681722d466f15660e1ded53888693484520,0,0,0,0,0,NaN
1,20200812,AL,104786.0,690985.0,NaN,1372.0,12292.0,NaN,1282.0,NaN,700.0,41523.0,NaN,8/12/2020 11:00,2020-08-12T11:00:00Z,08/12 07:00,1882.0,12292.0,2020-08-12T11:00:00Z,791786.0,NaN,NaN,100801.0,1814.0,68.0,1,935,11147,795771,795771,12082,795771,35,222,3b8e75d37adb9ba3951ec8b2aae85ba1fa9b02f1,0,0,0,0,0,NaN
2,20200812,AR,51114.0,522457.0,NaN,486.0,3472.0,NaN,NaN,113.0,455.0,42998.0,NaN,8/12/2020 14:40,2020-08-12T14:40:00Z,08/12 10:40,566.0,3472.0,2020-08-12T14:40:00Z,572868.0,NaN,522457.0,51114.0,NaN,NaN,5,703,0,573571,573571,703,573571,0,71,614656008210925b859900619d0426d793e79029,0,0,0,0,0,NaN
3,20200812,AS,0.0,1396.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7/24/2020 00:00,2020-07-24T00:00:00Z,07/23 20:00,0.0,NaN,2020-07-24T00:00:00Z,NaN,NaN,NaN,0.0,NaN,NaN,60,0,0,1396,1396,0,1396,0,0,f6988932e32e01718e47c8647131e3311e70d481,0,0,0,0,0,NaN
4,20200812,AZ,189443.0,854785.0,NaN,1469.0,19821.0,519.0,NaN,328.0,NaN,26418.0,NaN,8/12/2020 00:00,2020-08-12T00:00:00Z,08/11 20:00,4347.0,19821.0,2020-08-12T00:00:00Z,1042838.0,NaN,NaN,188053.0,4095.0,252.0,4,706,6247,1044228,1044228,6953,1044228,148,529,83481965d3dca76744e02ade2b8ec71c6fec31b7,0,0,0,0,0,NaN
